# Improvement Heuristics

## TSP

In [ ]:
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import numpy as np
import random


#Section 1 - MAIN FUNCTION: will use list of functions in #Section 2. Input neccessary parameters. Output best tour & graph.
# Parameters will be noted in Sections 2.

def MMAS_algo(N, Seed, nr_iterations, alpha, beta, nr_ants, rho, tau_min, tau_max):
    

# A. Get basic data, use functions A1,A2,A3. 
# Output: pos as positions, dist as distances, phero as pheromone matrix, prob_ma as probability matrix
    pos, dist = create_TSP_data(N, Seed)                                
    phero = pheromone_matrix(dist)
    prob_ma = probability_matrix(phero, dist, alpha, beta)              

# B. Main loop with number of iteration. Inside each iteration contains loop with number of ants. 
# The process: e.g In iteration 1, 5 ants make 5 tours based on probability matrix. Each of 5 tours will be imrpoved by LS.
# Each improved tour will be compared with current global best tour and update. 
    global_best_tour = 0
    best_length = 1000000                                         #best_length will be updated after each LS
    length_plot_y = []
    
    improve_dummy = 1000000
    improvement_plot_y = []
    improve_points = []
    
    for i in range(nr_iterations):                                                                      #loop 1 starts
        
        iteration_lengthlist = []   # list of all length 
        
        for j in range(nr_ants):                                                                        #loop 2 starts
            each_tour = find_tour_each_ant(dist, prob_ma)                               # apply B1
            afterLS_tour = tsp_improvement(each_tour, pos)                              # apply B3
            #print('Ant number', j+1, 'LS tour & length:', afterLS_tour,'-' ,tsp_tourlength(afterLS_tour,pos))

            if tsp_tourlength(afterLS_tour,pos) <= best_length:                         # apply B2 to update best_length
                best_length = tsp_tourlength(afterLS_tour,pos) 
                global_best_tour = afterLS_tour
            #print('*For ant number', j+1 ,'Best global tour & length now:',global_best_tour, '-',tsp_tourlength(global_best_tour, pos), '\n')
            
# C. Update pheromone matrix, probability matrix
            phero = pheromone_update(global_best_tour, pos, phero, rho, tau_min, tau_max)     # apply C
            prob_ma = probability_matrix(phero, dist, alpha, beta)                            # apply A3
        #print('**After Interation number',i+1,'updated global best tour & length:', global_best_tour,'-', tsp_tourlength(global_best_tour, pos) ,'\n')                                                                  

# D. Visualization     
# D1 Improvement graph
            iteration_lengthlist.append(tsp_tourlength(afterLS_tour,pos))
                                                                                                        #loop 2 ends
        sorted_lengthlist = sorted(iteration_lengthlist)                  # prepare length_plot_y
        best_length_iteration = sorted_lengthlist[0]            
        length_plot_y.append(best_length_iteration)
        
        if best_length_iteration <= improve_dummy:                        # prepare improvement_plot_y, improved points  
            improve_dummy = best_length_iteration
            improvement_plot_y.append(improve_dummy)
            improve_points.append(i+1) 
                                                                                                        #loop 1 ends
    graph_improvement(length_plot_y,improvement_plot_y, nr_iterations, improve_points)  #apply D1
                                                                       
# D2 Best global tour graph                                                                       
    #plot_TSPresult(pos, global_best_tour)                                              #apply D2
    
    return global_best_tour

        
#Section 2 - List of all functions

# A1. Create random 2d-points and a distance matrix
# Input: N as number of nodes, Seed for numpy random. Output: positions and distances
def create_TSP_data(N , seed):
    np.random.seed(seed)
    positions = np.random.rand(N, 2)
    distances = squareform(pdist(positions, 'euclidean'))
    return positions,distances

# A2. Preparation of pheromone matrix
# Input: distances from A1. Output: pheromone matrix
def pheromone_matrix(distances):
    pheromone_matrix = np.ones(distances.shape)           # pheromone matrix with initial max value = 1
    for i in range(len(distances)):                       # format with 0 for diagonal line
        pheromone_matrix[i][i] = 0
    return pheromone_matrix

# A3. Preparation of probability matrix
# Input pheromone matrix from A2 or C, distances from A1, alpha as weight of pheromone, beta as weight of distance heuristic
def probability_matrix(pheromone_matrix, distances, alpha, beta):
    eta = 1 / (distances + 0.01)
    probability_matrix = pheromone_matrix ** alpha * eta ** beta 
    return probability_matrix
# print('pheromone \n',pheromone)
# print('probability_matrix \n',probability_matrix)


# B1. Find tour of an ant, based on weighted probability (pheromone & distance heuristic)
# Input distances from A1, probability matrix from A3
def find_tour_each_ant(distances, probability_matrix):  
    tour = [0]
    prev = 0                                                  # assumption : start at node 0
    visited_list = []
    local_probability_matrix = np.copy(probability_matrix)
    
    for i in range(len(distances) - 1):                       # use random with weight
        move = random.choices(range(len(local_probability_matrix)), weights = local_probability_matrix[prev], k = 1)[0]
        visited_list.append(prev)
        prev = move
        visited_list.append(move)
        local_probability_matrix[:, visited_list] = 0         # set columns of visited node to 0
        tour.append(move)
    return tour


# B2.Find tour length. Input tour can be from any tour found or improved. Ourput the length of a tour.
def tsp_tourlength(tour, positions):
    tour_distance = 0
    for i in range(len(tour)):
        start_pos = positions[tour[i - 1]]
        end_pos = positions[tour[i]]
        tour_distance += np.linalg.norm(end_pos - start_pos)
    return tour_distance


# B3. Local search improvement heuristic (first improvement, adjacent swap)
# Input tour of each ant and positions of nodes. 
def tsp_improvement(tour, positions):
    length = tsp_tourlength(tour, positions)
    #print("Seed tour & length:",tour, '-',length)
    for i in range(len(tour)):
        tour[i-1],tour[i] = tour[i],tour[i-1]
        if tsp_tourlength(tour, positions) < length:
            length = tsp_tourlength(tour,positions)
            #print('Found improved tour + length:', tour,'-' ,length)
            return tour
        else:
             tour[i-1],tour[i] = tour[i],tour[i-1]
             #print("found no improvement")
    return tour


# C. Pheromone update with 3 stages: 1-deposit, 2-evaporate, 3-check max-min limit
# Input global best tour after each update, positions, current pheromone matrix, rho as evaporation rate, 
# tau_min,max as limit of pheromone trail  
def pheromone_update(tour, positions, pheromone_matrix, rho, tau_min, tau_max):
    #print('previous pheromone matrix', pheromone_matrix)
    for i in range(len(tour)):                                           #1 start to deposit
        node1 = tour[i-1]
        node2 = tour[i]
        node12_distance = np.linalg.norm(positions[node2] - positions[node1])
        delta = 0.1 / node12_distance                      # deposit by adding delta = 0.1/distance between 2 nodes
        pheromone_matrix[node1][node2] += delta
        pheromone_matrix[node2][node1] = pheromone_matrix[node1][node2]
    #print('deposited phero matrix \n', pheromone_matrix)

    pheromone_matrix = pheromone_matrix * (1.0 - rho)                    #2 evaporate with rho

    pheromone_matrix = np.clip(pheromone_matrix, tau_min, tau_max)       #3 check max min pheromone

    for i in range(len(tour)):                                           # final format with 0 for diagonal line
        pheromone_matrix[i][i] = 0
    return pheromone_matrix
    

# D. Visualization
   
# D1 Improvement graph
# length_plot_y: list of length of all iterations (e.g 50 iteration -> 50 lengths). It corresponds with nr_iterations
# improvement_plot_y: list of length of improvement points. It corresponds with improve_points
def graph_improvement(length_plot_y,improvement_plot_y,nr_iterations, improve_points):
    
    x = list(range(1, nr_iterations+1))
    x1 = improve_points
    y = length_plot_y 
    y1 = improvement_plot_y
    plt.figure()
    plt.plot(x, y, '--bo', label="Interation best")
    plt.plot(x1, y1, color ='red', label="Improvement line")
    plt.legend(loc="upper right")
    plt.grid()
    plt.axhline(min(y), color ='green', lw = 1)
    plt.xlabel("Iterations")
    plt.ylabel("Tour length")
    plt.title(" Improvement of best solution ")
    plt.show()


# D2 Tour graph
def plot_TSPresult(positions, tour):
    fig, ax = plt.subplots(2, sharex=True, sharey=True)         # Prepare 2 plots
    ax[0].set_title('Raw nodes')
    ax[1].set_title('Optimized tour')
    ax[0].scatter(positions[:, 0], positions[:, 1])             # plot A
    ax[1].scatter(positions[:, 0], positions[:, 1])             # plot B
    distance = 0.
    for i in range(len(tour)):
        start_pos = positions[tour[i-1]]
        end_pos = positions[tour[i]]
        ax[1].annotate("",
            xy=start_pos, xycoords='data',
            xytext=end_pos, textcoords='data',
            arrowprops=dict(arrowstyle="<-",
                            connectionstyle="arc3"))
        ax[1].annotate(str(tour[i-1]),xy=start_pos, xytext=start_pos+0.01)
        distance += np.linalg.norm(end_pos - start_pos)
    textstr = "Total length: %.3f" % (distance)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax[1].text(0.05, 0.95, textstr, transform=ax[1].transAxes, fontsize=12, #  Textbox
            verticalalignment='top', bbox=props)
    plt.tight_layout()
    plt.show()

In [ ]:
'''
Note: 
If the graph and results of each iteration are not shown, re-comment the print statement in the main algorigthm. 
They are commented out to save time for test of parameters.
'''

#MMAS_algo(N =5, Seed = 101, nr_iterations = 50, alpha = 1, beta =1, nr_ants = 5, rho = 0.01, tau_min = 0.1, tau_max = 1)
#MMAS_algo(N =10, Seed = 53, nr_iterations = 50, alpha = 1, beta =1, nr_ants = 5, rho = 0.01, tau_min = 0.1, tau_max = 1)
MMAS_algo(N =20, Seed = 99, nr_iterations = 50, alpha = 1, beta =1, nr_ants = 5, rho = 0.01, tau_min = 0.1, tau_max = 1)
#MMAS_algo(N =100, Seed = 12, nr_iterations = 100, alpha = 1, beta =1, nr_ants = 5, rho = 0.01, tau_min = 0.1, tau_max = 1)

In [ ]:
'''
Function to find optimal value for 4 parameters: beta, rho, tau_min, tau_max. Others are pre-determined.
How it works:
- Loop through nested loops of 4 parameters, find the best optimal of lower loop and then apply to upper loop. 
Input: 
- beta_range is the range of beta that users want to test, e.g 10 means test beta from 1 to 10,
- var_step is step of range for rho, tau_min, tau_max, 
 e.g 5 means: rho range (0,1) returns array of 5 values: 0 - 0.25 - 0.5 - 0.75 - 1
- rho/tau_min/tau_max: the array of value in range with steps as explained above
Output:
- Optimal value of testes parameters
'''
#Section 3 - Parameters test

# E1 Nested loops of parameters test. Can be extended to more parameters such as alpha, nr_ants by futher modification.
def test(beta_range,var_step, rho, tau_min, tau_max):
    N, Seed, nr_iterations, alpha, nr_ants, = 10, 101, 5, 1, 5
    pos, dist = create_TSP_data(N, Seed)
    
    tourandrho_list = []
    tourandbeta_list = []
    tourandtaumin_list = []
    tourandtaumax_list = []

    for h in range(1,var_step+1):         #loop tau_max
        for k in range(1,var_step+1):       #loop tau_min     
            for i in range(1,beta_range+1):  #loop beta          
                for j in range(1,var_step+1):  #loop rho
                    
                    tours_rho = MMAS_algo(N, Seed, nr_iterations, alpha, i, nr_ants, rho[j-1], tau_min[k-1], tau_max[h-1])  #i = beta, rho = rho[j-1]
                    tourandrho_list.append((rho[j-1], tsp_tourlength(tours_rho,pos)))
                sorted_rho_list = sorted(tourandrho_list, key=lambda x: x[1])
                best_rho = tourandrho_list[0][0]
                #print('*level 1','taumin =', tau_min[k-1], 'beta =',i,'best_rho =', best_rho)

                tours_beta = MMAS_algo(N, Seed, nr_iterations, alpha, i, nr_ants, best_rho, tau_min[k-1], tau_max[h-1])  #rho = best_rho, tau_min = tau_min[k-1]
                tourandbeta_list.append((i, tsp_tourlength(tours_beta,pos)))
            sorted_beta_list = sorted(tourandbeta_list, key=lambda x: x[1])
            best_beta = sorted_beta_list[0][0]
            #print('*sorted_beta_list of beta nr',sorted_beta_list)
            #print('#level 2','taumin =', tau_min[k-1], 'best_beta =', best_beta,'best_rho =', best_rho ,'\n')
                                                                                     #rho = best_rho, tau_min = tau_min[k-1]
            tours_taumin = MMAS_algo(N, Seed, nr_iterations, alpha, best_beta, nr_ants, best_rho, tau_min[k-1], tau_max[h-1])  
            tourandtaumin_list.append((tau_min[k-1], tsp_tourlength(tours_taumin,pos)))        
        sorted_taumin_list = sorted(tourandtaumin_list, key=lambda x: x[1])
        best_taumin = sorted_taumin_list[0][0]
        #print('*sorted_taumin_list of var value',sorted_taumin_list)
        #print('¤level 3','**best_taumin =', best_taumin, 'best_beta =', best_beta,'best_rho =', best_rho, '\n')

        tours_taumax = MMAS_algo(N, Seed, nr_iterations, alpha, best_beta, nr_ants, best_rho, best_taumin, tau_max[h-1])  
        tourandtaumax_list.append((tau_max[h-1], tsp_tourlength(tours_taumax,pos)))        
    sorted_taumax_list = sorted(tourandtaumax_list, key=lambda x: x[1])
    best_taumax = sorted_taumax_list[0][0]
    #print('*sorted_taumin_list of var value',sorted_taumin_list)
    #print('#level 4','best_taumax =', best_taumax,'best_taumin =', best_taumin,'best_beta =', best_beta,'best_rho =', best_rho, '\n')
    return best_taumax, best_taumin, best_beta, best_rho

'''
Users now input the range of parameters of MMAS ACO to test. Explaination are below: 
'''
# E2 User interface (basic version without error handling)
def User_parameters_test():
    
    beta_range = int(input('Input Beta Range as integer only!, e.g 10 if you want to test beta 1 to 10: '))
    var_step = int(input('Input Steps of the range of tau_min, tau_max, rho as integer only!: '))
    a = float(input('Input min value for range test of rho evaporation here, e.g 0.005: '))
    b = float(input('Input max value for range test of rho evaporation here, e.g 0.2: '))
    rho = np.linspace(a, b, num= var_step)
    c = float(input('Input min value for range test of tau_min here, e.g 0.05: '))
    d = float(input('Input max value for range test of tau_min here, e.g 0.4: '))
    tau_min = np.linspace(c, d, num= var_step)
    x = float(input('Input min value for range test of tau_max here, e.g 0.8: '))
    y = float(input('Input max value for range test of tau_max here, e.g 2: '))
    tau_max = np.linspace(x, y, num= var_step)
    
    print('Please wait, the test can take up to 1 minute or more')
    best_taumax, best_taumin, best_beta, best_rho = test(beta_range,var_step, rho, tau_min, tau_max)
    print('Best_taumax =', best_taumax,', Best_taumin =', best_taumin,', Best_beta =', best_beta,', Best_rho =', best_rho, '\n')

User_parameters_test()
